![python image2](https://wikidocs.net/images/page/22886/rnn_image2_ver3.PNG)

# 1. RNN?
  1. 내부적으로 순환(Recurrent)되는 구조를 이용하여
  2. 순서(Sequence)가 있는 데이터를 처리하는 데 강점을 가진 NN  

## 1.1 순서가 있는 데이터
- 문장이나 음성, 시계열 데이터처럼 Sequence에 따른 데이터
- 예를 들어, I work at google / I google at work 두 문장이 있을 때, 동사로 활용된 work 또는 google을 동사로 제대로 해석하려면 첫 번째 단어가 주어 'I'라는 것을 메모리 상에 기억하고 있어야함. 또한 I, work(or google)까지 기억한다면 at google과 at work를 올바르게 해석하여 결국 문장 전체를 올바르게 해석할 수 있음.  
<br><br>
  
**RNN은**
  1. 은닉층 내의 순환구조를 이용하여 메모리 cell이 각 sequence의 데이터를 기억하고 있다가, 
  2. 새롭게 입력으로 주어지는 데이터(입력)와 메모리 cell이 옆 층으로 전달해준 과거의(t-1로 가정) 데이터를 연결시켜 출력값(의미)을 알아내는 기능을 가짐

## 1.2 DNN Review

![python image2](image_source/DNN_principle.png)

## 1.3. RNN 구조

![python image2](image_source/RNN_architecture.png)

## 1.4 RNN 동작 원리 - 1

![python image2](image_source/RNN_principle_00.png)

![python image2](image_source/RNN_principle_01.png)

1. 'I'의 품사: (t1) 참고할만한 과거의 data가 없어서
2. 'work'의 품사: (t2) 은닉층으로 들어오긴 하는데, t1시점의 출력값을 I(주어)를 우리가 기억하고 있으므로 I를 활용해 은닉층 계산을 하고 출력층에서 가장 확률이 높은 품사 기준으로 값 출력
3. 'at'의 품사: (t3) t1 시점의 I, 그리고 t1시점 I의 영향을 받은 t2 시점의 work 출력값을 기준(softmax)으로 출력한다.
4. 'google'의 품사: (t4) t1, t2, t3 I, work, at를 토대로 주어, 동사, 전치사 다음에는 명사가 올 확률이 높기 때문에 softmax기준으로 명사로 분류


## 1.5 RNN 동작원리 - 2
![python image2](image_source/RNN_principle_02.png)

첫 번째 데이터라 $H_{cur} = 0$

## 1.6 RNN 동작원리 - 3

![python image2](image_source/RNN_principle_03.png)

Embedding을 사용하며, 단어 집합(Vocabulary)의 크기가 5,000이고 임베딩 벡터의 차원은 100입니다.
은닉층에서는 Simple RNN을 사용하며, 은닉 상태의 크기는 128입니다.
훈련에 사용하는 모든 샘플의 길이는 30으로 가정합니다.
이진 분류를 수행하는 모델로, 출력층의 뉴런은 1개로 시그모이드 함수를 사용합니다.
은닉층은 1개입니다.


In [1]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Embedding, Dense

vocab_size = 5000
embedding_dim = 100
hidden_size = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_size))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         500000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               29312     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 529,441
Trainable params: 529,441
Non-trainable params: 0
_________________________________________________________________


In [2]:
5000 * 100 + 100*128 + 128*128 + 128 + 128 + 1

529441

# 2. 장단기 메모리(Long Short-Term Memory, LSTM)
- 앞서 배운 RNN(vanilla)의 한계를 극복하기 위한 변형

![python image](https://wikidocs.net/images/page/22888/lstm_image1_ver2.PNG)

음영을 정보량으로 가정했을 때 1에서 t로 갈수록 음영이 옅어지는 것을 볼 수 있음. 이는 곧 과거의 정보량이 점차 소실되어가는 과정을 표현한 것.

예)
  - 모스크바에 여행을 왔는데, 건물도 예쁘고 음식도 맛있었어. 그런데 직장 상사한테 전화가 왔어 어디냐고 묻더라고~ (중량)
  - 만약 이 문장을 임베딩해서 다음 단어를 예측한다라고 했을 때, 모스크바라는 벡터의 정보량이 소실된다면 예측에 어떤 영향을 줄 것인가?
  
 ***= 장기 의존성 문제(the problem of Long-Term Dependencies)***
 
 
 
**그림이 뭔가 어렵네요!**
 ![python image](https://wikidocs.net/images/page/22888/vaniila_rnn_and_different_lstm_ver2.PNG)
 
 
 각각의 요소에 대한 설명도 읽어보았으나... 넘어가야할 것 같네요...

# 3. 게이트 순환 유닛(Gated Recurrent Unit, GRU)
- GRU는 LSTM의 장기 의존성 문제에 대한 해결책을 유지하면서, 은닉 상태를 업데이트하는 계산을 줄였습니다. 다시 말해서, GRU는 성능은 LSTM과 유사하면서 복잡했던 LSTM의 구조를 간단화 시켰습니다.

![python image2](https://wikidocs.net/images/page/22889/GRU.PNG)

  
- LSTM에서는 출력, 입력, 삭제 게이트라는 3개의 게이트가 존재했습니다. 반면, GRU에서는 업데이트 게이트와 리셋 게이트 두 가지 게이트만이 존재합니다.
- 데이터 양이 적을 때는, 매개 변수의 양이 적은 GRU가 조금 더 낫고, 데이터 양이 더 많으면 LSTM이 더 낫다고 알려져 있습니다.

# 4. 케라스의 SimpleRNN과 LSTM 이해하기

## 4.1 임의의 입력 생성하기

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional
import pprint

In [3]:
# 우선 RNN과 LSTM을 테스트하기 위한 임의의 입력을 만듭니다.

train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
pprint.pprint(train_X)
print(np.shape(train_X))
print("5개의 단어 벡터를 사용한 4번의 기록(시점)")

[[0.1, 4.2, 1.5, 1.1, 2.8],
 [1.0, 3.1, 2.5, 0.7, 1.1],
 [0.3, 2.1, 1.5, 2.1, 0.1],
 [2.2, 1.4, 0.5, 0.9, 1.1]]
(4, 5)
5개의 단어 벡터를 사용한 4번의 기록(시점)


In [4]:
# 앞서 RNN은 2D 텐서가 아니라 3D 텐서를 입력을 받는다고 언급한 바 있습니다. 
# 즉, 위에서 만든 2D 텐서를 3D 텐서로 변경합니다. 이는 배치 크기 1을 추가해주므로서 해결합니다.
# 한 번 더 []로 싸서 차원추가
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)
#(batch_size(timesteps와 input 한 세트를 몇 개씩 넣을 것인지), timesteps: sequence legnth(Series의 길이), input_dim: 입력하는 데이터의 차원(5개니까 5))

(1, 4, 5)


![python image2](image_source/rnn_hello.png)

![python image2](image_source/rnn_shape.png)

## 4.2 SimpleRNN 이해하기

In [9]:
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일. 
# return sequence = RNN 계산과정 상에 있는 hidden state를 출력할 것인가(== 마지막만 출력할래?), To one, To many를 결정

hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[0.9334026  0.00360336 0.9851538 ]], shape: (1, 3)


In [10]:
# 기본적으로 return_sequences가 False인 경우에는 SimpleRNN은 마지막 시점의 은닉 상태만 출력합니다. == To one
# 이번에는 return_sequences를 True로 지정하여 모든 시점의 은닉 상태를 출력해봅시다.== time step별 hidden state를 모두 출력 == To many 

rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

print('hidden states \n: {}, shape: {}'.format(hidden_states, hidden_states.shape))
# (batch_size, timesteps, input_dim)
# ()

hidden states 
: [[[ 0.35987827  0.9998444   0.9696535 ]
  [-0.7668005   0.98996025  0.99939626]
  [-0.8133425   0.6670614   0.98648685]
  [-0.9896994   0.6764782   0.9983962 ]]], shape: (1, 4, 3)


In [11]:
# 그렇다면 return_sequences는 False인데, retun_state가 True인 경우는 어떨까요?
# return state = LSTM 레이어에서 주로 사용되는 값
# 마지막 time step에서의 output(hidden state), hidden state와 cell state가 출력된다. 
# 즉 마지막 output값이 2번 출력이 되고 cell state가 나온다.

rnn = SimpleRNN(3, return_sequences=False, return_state=True)
hidden_state, last_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

print("두 개의 출력 모두 마지막 시점의 은닉 상태를 출력하게 됩니다.")

hidden state : [[ 0.9517502  -0.52132624  0.9982433 ]], shape: (1, 3)
last hidden state : [[ 0.9517502  -0.52132624  0.9982433 ]], shape: (1, 3)
두 개의 출력 모두 마지막 시점의 은닉 상태를 출력하게 됩니다.


## 4.3 LSTM 이해하기

- 실제로 SimpleRNN이 사용되는 경우는 거의 없습니다.
- 이보다는 LSTM이나 GRU을 주로 사용하는데, 이번에는 임의의 입력에 대해서 LSTM을 사용할 경우를 보겠습니다. 우선 return_sequences를 False로 두고, return_state가 True인 경우를 봅시다.

In [12]:
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_state, last_state, last_cell_state = lstm(train_X)



print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden state : [[-0.20158324 -0.61441725  0.11092292]], shape: (1, 3)
last hidden state : [[-0.20158324 -0.61441725  0.11092292]], shape: (1, 3)
last cell state : [[-0.5716053  -1.4566205   0.19124356]], shape: (1, 3)


- 이번에는 SimpleRNN 때와는 달리, 세 개의 결과를 반환합니다. return_sequences가 False이므로 우선 첫번째 결과는 마지막 시점의 은닉 상태입니다. 그런데 LSTM이 SimpleRNN과 다른 점은 return_state를 True로 둔 경우에는 마지막 시점의 은닉 상태뿐만 아니라 셀 상태까지 반환한다는 점입니다. 이번에는 return_sequences를 True로 바꿔보겠습니다.

In [13]:
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_hidden_state, last_cell_state = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_hidden_state, last_hidden_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden states : [[[ 0.30427808 -0.0732515   0.2798049 ]
  [ 0.39632452 -0.17029493  0.2101612 ]
  [ 0.5335901  -0.11439174  0.29666686]
  [ 0.60747755 -0.45178524  0.43473914]]], shape: (1, 4, 3)
last hidden state : [[ 0.60747755 -0.45178524  0.43473914]], shape: (1, 3)
last cell state : [[ 0.99897736 -0.8454615   0.6353362 ]], shape: (1, 3)


return_state가 True이므로 두번째 출력값이 마지막 은닉 상태, 세번째 출력값이 마지막 셀 상태인 것은 변함없지만 return_sequences가 True이므로 첫번째 출력값은 모든 시점의 은닉 상태가 출력됩니다.

## 4.4 Bidirectional(LSTM) 이해하기(양방향순환)



난이도를 조금 올려서 양방향 LSTM의 출력값을 확인해보겠습니다. return_sequences가 True인 경우와 False인 경우에 대해서 은닉 상태의 값이 어떻게 바뀌는지 직접 비교하기 위해서 이번에는 출력되는 은닉 상태의 값을 고정시켜주겠습니다.

In [14]:
k_init = tf.keras.initializers.Constant(value=0.1) # kernel
b_init = tf.keras.initializers.Constant(value=0) # bias
r_init = tf.keras.initializers.Constant(value=0.1) # recurrent

우선 return_sequences가 False이고, return_state가 True인 경우입니다.

In [15]:
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[0.6303138 0.6303138 0.6303138 0.7038734 0.7038734 0.7038734]], shape: (1, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


이번에는 무려 5개의 값(hidden_states, forward_h, forward_c, backward_h, backward_c)을 반환합니다. return_state가 True인 경우에는 정방향 LSTM의 은닉 상태와 셀 상태, 역방향 LSTM의 은닉 상태와 셀 상태 4가지를 반환하기 때문입니다. 다만, 셀 상태는 각각 forward_c와 backward_c에 저장만 하고 출력하지 않았습니다. 첫번째 출력값의 크기가 (1, 6)인 것에 주목합시다. 이는 return_sequences가 False인 경우 정방향 LSTM의 마지막 시점의 은닉 상태와 역방향 LSTM의 첫번째 시점의 은닉 상태가 연결된 채 반환되기 때문입니다. 그림으로 표현하면 아래와 같이 연결되어 다음층에서 사용됩니다.

![python image2](https://wikidocs.net/images/page/94748/bilstm3.PNG)

마찬가지로 return_state가 True인 경우에 반환한 은닉 상태의 값인 forward_h와 backward_h는 각각 정방향 LSTM의 마지막 시점의 은닉 상태와 역방향 LSTM의 첫번째 시점의 은닉 상태값입니다. 그리고 이 두 값을 연결한 값이 hidden_states에 출력되는 값입니다.

이를 이용한 실습은 11챕터의 BiLSTM으로 한국어 스팀 리뷰 분류하기(https://wikidocs.net/94748)에 준비되어 있습니다.

정방향 LSTM의 마지막 시점의 은닉 상태값과 역방향 LSTM의 첫번째 은닉 상태값을 기억해둡시다.

- 정방향 LSTM의 마지막 시점의 은닉 상태값 : [0.6303139 0.6303139 0.6303139]
- 역방향 LSTM의 첫번째 시점의 은닉 상태값 : [0.70387346 0.70387346 0.70387346]  

현재 은닉 상태의 값을 고정시켜두었기 때문에 return_sequences를 True로 할 경우, 출력이 어떻게 바뀌는지 비교가 가능합니다.

In [12]:
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.55111325 0.55111325 0.55111325 0.58863586 0.58863586 0.58863586]
  [0.59115744 0.59115744 0.59115744 0.3951699  0.3951699  0.3951699 ]
  [0.6303138  0.6303138  0.6303138  0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


hidden states의 출력값에서는 이제 모든 시점의 은닉 상태가 출력됩니다. 역방향 LSTM의 첫번째 시점의 은닉 상태는 더 이상 정방향 LSTM의 마지막 시점의 은닉 상태와 연결되는 것이 아니라 정방향 LSTM의 첫번째 시점의 은닉 상태와 연결됩니다.

그림으로 표현하면 다음과 같이 연결되어 다음층의 입력으로 사용됩니다.

![python image2](https://wikidocs.net/images/page/94748/bilstm1.PNG)

# 5. RNN 언어 모델(Recurrent Neural Network Language Model, RNNLM)

앞서 n-gram 언어 모델과 NNLM은 고정된 개수의 단어만을 입력으로 받아야한다는 단점이 있었습니다. 하지만 시점(time step)이라는 개념이 도입된 RNN으로 언어 모델을 만들면 입력의 길이를 고정하지 않을 수 있습니다. 이처럼 RNN으로 만든 언어 모델을 RNNLM(Recurrent Neural Network Language Model)이라고 합니다.




아래 그림은 RNNLM이 어떻게 이전 시점의 단어들과 현재 시점의 단어로 다음 단어를 예측하는지를 보여줍니다.
(주어진 단어 시퀀스로부터 다음 단어를 예측하는 모델)

**예문 : "what will the fat cat sit on"**



![python image2](https://wikidocs.net/images/page/46496/rnnlm1_final_final.PNG)

RNNLM은 기본적으로 예측 과정에서 이전 시점의 출력을 현재 시점의 입력으로 합니다. RNNLM은 what을 입력받으면, will을 예측하고 이 will은 다음 시점의 입력이 되어 the를 예측합니다. (h, e, l, o voca를 가지고 있을 때, h -> e -> l -> l -> o를 예측하는 식)

**사실 위 과정은 훈련이 끝난 모델의 테스트 과정 동안(실제 사용할 때)의 이야기입니다.** 훈련 과정에서는 이전 시점의 예측 결과를 다음 시점의 입력으로 넣으면서 예측하는 것이 아니라, what will the fat cat sit on라는 훈련 샘플이 있다면, what will the fat cat sit 시퀀스를 모델의 입력으로 넣으면, will the fat cat sit on를 예측하도록 훈련됩니다. will, the, fat, cat, sit, on는 각 시점의 레이블입니다.

RNN 훈련 기법을 **교사 강요(teacher forcing)**라고 합니다.  

**교사 강요(teacher forcing)란,**  
**테스트 과정**에서 t 시점의 출력이 t+1 시점의 입력으로 사용되는 RNN 모델을 훈련시킬 때 사용하는 훈련 기법입니다.  
**훈련할 때** 교사 강요를 사용할 경우, **모델이 t 시점에서 예측한 값을 t+1 시점에 입력으로 사용하지 않고, t 시점의 레이블. 즉, 실제 알고있는 정답을 t+1 시점의 입력으로 사용**합니다.  

훈련 과정에서도 이전 시점의 출력을 다음 시점의 입력으로 사용하면서 훈련 시킬 수도 있지만 이는 한 번 잘못 예측하면 뒤에서의 예측까지 영향을 미쳐 훈련 시간이 느려지게 되므로 교사 강요를 사용하여 RNN을 좀 더 빠르고 효과적으로 훈련시킬 수 있습니다.

**activation function: softmax  
loss function: cross entropy**

![python image2](https://wikidocs.net/images/page/46496/rnnlm2_final_final.PNG)

RNN 구조 

총 4개의 층(layer)으로 이루어진 인공 신경망입니다. 우선 입력층(input layer)을 봅시다. RNNLM의 현 시점(timestep)은 4로 가정합니다. 그래서 4번째 입력 단어인 fat의 원-핫 벡터가 입력이 됩니다.

이제 출력층(Output layer)를 봅시다. 모델이 예측해야하는 정답에 해당되는 단어 cat의 원-핫 벡터는 출력층에서 모델


![python image2](https://wikidocs.net/images/page/46496/rnnlm3_final.PNG)


![python image2](https://wikidocs.net/images/page/46496/rnnlm4_final.PNG)

***NNLM 투사층**

![python image2](https://wikidocs.net/images/page/45609/nnlm1.PNG)

현 시점의 입력 단어의 원-핫 벡터 $x_{t}$를 입력 받은 RNNLM은 우선 임베딩층(embedding layer)을 지납니다. 이 임베딩층은 기본적으로 NNLM 챕터에서 배운 투사층(projection layer)입니다. NNLM 챕터에서는 룩업 테이블을 수행하는 층을 투사층라고 표현했지만, 이미 투사층의 결과로 얻는 벡터를 임베딩 벡터라고 부른다고 NNLM 챕터에서 학습하였으므로, 앞으로는 임베딩 벡터를 얻는 투사층을 임베딩층(embedding layer)이라는 표현을 사용할 겁니다.

단어 집합의 크기가 V일 때, 임베딩 벡터의 크기를 M으로 설정하면, 각 입력 단어들은 임베딩층에서 V × M 크기의 임베딩 행렬과 곱해집니다. 여기서 V는 단어 집합의 크기를 의미합니다. 만약 원-핫 벡터의 차원이 7이고, M이 5라면 임베딩 행렬은 7 × 5 행렬이 됩니다. 그리고 이 임베딩 행렬은 역전파 과정에서 다른 가중치들과 함께 학습됩니다. 이는 NNLM 챕터에서 이미 배운 개념입니다.

임베딩층: $e_{t} = lookup(x_{t})$

이 임베딩 벡터는 은닉층에서 이전 시점의 은닉 상태인 $h_{t-1}$과 함께 다음의 연산을 하여 현재 시점의 은닉 상태 $h_{t}$를 계산하게 됩니다

은닉층: $h_{t} = tanh(W_{x} e_{t} + W_{h}h_{t−1} + b)$

출력층에서는 활성화 함수로 소프트맥스(softmax) 함수를 사용하는데, V차원의 벡터는 소프트맥스 함수를 지나면서 각 원소는 0과 1사이의 실수값을 가지며 총 합은 1이 되는 상태로 바뀝니다. 이렇게 나온 벡터를 RNNLM의 t시점의 예측값이라는 의미에서 $\hat{y_{t}}$
라고 합시다. 이를 식으로 표현하면 아래와 같습니다.

출력층: $\hat{y_{t}} = softmax(W_{y}h_{t} + b)$

벡터 $\hat{y_{t}}$의 각 차원 안에서의 값이 의미하는 것은 이와 같습니다. 
$\hat{y_{t}}$의 j번째 인덱스가 가진 0과 1사이의 값은 j번째 단어가 다음 단어일 확률을 나타냅니다. 그리고 $\hat{y_{t}}$
는 실제값. 즉, 실제 정답에 해당되는 단어인 원-핫 벡터의 값에 가까워져야 합니다. 실제값에 해당되는 다음 단어를 라고 했을 때, 이 두 벡터가 가까워지게 하기위해서 RNNLM는 손실 함수로 cross-entropy 함수를 사용합니다. 그리고 역전파가 이루어지면서 가중치 행렬들이 학습되는데, 이 과정에서 임베딩 벡터값들도 학습이 됩니다.

룩업 테이블의 대상이 되는 테이블인 임베딩 행렬을 라고 하였을 때, 결과적으로 RNNLM에서 학습 과정에서 학습되는 가중치 행렬은 다음의  $E, W_{x}, W_{h}, W_{y}$ 4개 입니다. 뒤의 글자 단위 RNN 챕터에서 RNN 언어 모델을 구현해보면서 훈련 과정과 테스트 과정의 차이를 이해해보겠습니다.

# 6. RNN을 이용한 텍스트 생성(Text Generation using RNN)

이번 챕터에서는 다 대 일(many-to-one) 구조의 RNN을 사용하여 문맥을 반영해서 텍스트를 생성하는 모델을 만들어봅시다.



## 6.1 RNN을 이용하여 텍스트 생성하기

예를 들어서 '경마장에 있는 말이 뛰고 있다'와 '그의 말이 법이다'와 '가는 말이 고와야 오는 말이 곱다'라는 세 가지 문장이 있다고 해봅시다. 모델이 문맥을 학습할 수 있도록 전체 문장의 앞의 단어들을 전부 고려하여 학습하도록 데이터를 재구성한다면 아래와 같이 총 11개의 샘플이 구성됩니다.

In [20]:
import pandas as pd
pd.DataFrame({"X":["경마장에"
, "경마장에 있는"
, "경마장에 있는 말이"
, "경마장에 있는 말이 뛰고"
, "그의"
, "그의 말이"
, "가는"
, "가는 말이"
, "가는 말이 고와야"
, "가는 말이 고와야 오는"
, "가는 말이 고와야 오는 말이"]
, "Y":["있는"
,"말이"
,"뛰고"
,"있다"
,"말이"
,"법이다"
,"말이"
,"고와야"
,"오는"
,"말이"
,"곱다"]})

,X,Y
0,경마장에,있는
1,경마장에 있는,말이
2,경마장에 있는 말이,뛰고
3,경마장에 있는 말이 뛰고,있다
4,그의,말이
5,그의 말이,법이다
6,가는,말이
7,가는 말이,고와야
8,가는 말이 고와야,오는
9,가는 말이 고와야 오는,말이


### 6.1.1 데이터에 대한 이해와 전처리

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

우선 예제로 언급한 3개의 한국어 문장을 저장합니다.



In [22]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""
print(text)

경마장에 있는 말이 뛰고 있다

그의 말이 법이다

가는 말이 고와야 오는 말이 곱다



단어 집합을 생성하고 크기를 확인해보겠습니다.

In [26]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [27]:
print(t.word_index)


{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [28]:
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [29]:
print(sequences)


[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


**위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다.** [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다. 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.

우선 전체 샘플에 대해서 길이를 일치시켜 줍니다. 가장 긴 샘플의 길이를 기준으로 합니다. 현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6입니다. 이를 코드로는 다음과 같이 구할 수 있습니다.

In [30]:
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [33]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') # padding = "pre" 앞을 채워서 길이 6 맞추기
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [34]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [35]:
X

array([[ 0,  0,  0,  0,  2],
       [ 0,  0,  0,  2,  3],
       [ 0,  0,  2,  3,  1],
       [ 0,  2,  3,  1,  4],
       [ 0,  0,  0,  0,  6],
       [ 0,  0,  0,  6,  1],
       [ 0,  0,  0,  0,  8],
       [ 0,  0,  0,  8,  1],
       [ 0,  0,  8,  1,  9],
       [ 0,  8,  1,  9, 10],
       [ 8,  1,  9, 10,  1]])

In [37]:
y

array([ 3,  1,  4,  5,  1,  7,  1,  9, 10,  1, 11])

In [38]:
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

# 여기부터 다시

### 6.1.2 모델 설계하기

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4753 - accuracy: 0.0909
Epoch 2/200
1/1 - 0s - loss: 2.4612 - accuracy: 0.0909
Epoch 3/200
1/1 - 0s - loss: 2.4473 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4336 - accuracy: 0.3636
Epoch 5/200
1/1 - 0s - loss: 2.4198 - accuracy: 0.3636
Epoch 6/200
1/1 - 0s - loss: 2.4060 - accuracy: 0.4545
Epoch 7/200
1/1 - 0s - loss: 2.3920 - accuracy: 0.4545
Epoch 8/200
1/1 - 0s - loss: 2.3777 - accuracy: 0.4545
Epoch 9/200
1/1 - 0s - loss: 2.3630 - accuracy: 0.5455
Epoch 10/200
1/1 - 0s - loss: 2.3479 - accuracy: 0.5455
Epoch 11/200
1/1 - 0s - loss: 2.3323 - accuracy: 0.6364
Epoch 12/200
1/1 - 0s - loss: 2.3161 - accuracy: 0.6364
Epoch 13/200
1/1 - 0s - loss: 2.2992 - accuracy: 0.5455
Epoch 14/200
1/1 - 0s - loss: 2.2816 - accuracy: 0.5455
Epoch 15/200
1/1 - 0s - loss: 2.2633 - accuracy: 0.5455
Epoch 16/200
1/1 - 0s - loss: 2.2441 - accuracy: 0.5455
Epoch 17/200
1/1 - 0s - loss: 2.2241 - accuracy: 0.5455
Epoch 18/200
1/1 - 0s - loss: 2.2032 - accuracy: 0.5455
E

Epoch 147/200
1/1 - 0s - loss: 0.1762 - accuracy: 1.0000
Epoch 148/200
1/1 - 0s - loss: 0.1721 - accuracy: 1.0000
Epoch 149/200
1/1 - 0s - loss: 0.1680 - accuracy: 1.0000
Epoch 150/200
1/1 - 0s - loss: 0.1642 - accuracy: 1.0000
Epoch 151/200
1/1 - 0s - loss: 0.1604 - accuracy: 1.0000
Epoch 152/200
1/1 - 0s - loss: 0.1567 - accuracy: 1.0000
Epoch 153/200
1/1 - 0s - loss: 0.1532 - accuracy: 1.0000
Epoch 154/200
1/1 - 0s - loss: 0.1497 - accuracy: 1.0000
Epoch 155/200
1/1 - 0s - loss: 0.1464 - accuracy: 1.0000
Epoch 156/200
1/1 - 0s - loss: 0.1431 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.1400 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.1369 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.1339 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.1311 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.1283 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.1256 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.1229 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

In [26]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [27]:
print(sentence_generation(model, t, '경마장에', 4))
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

C:\Users\mrjjo\anaconda3\envs\nlp38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


경마장에 있는 말이 뛰고 있다


In [28]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측


그의 말이 법이다


In [31]:
print(sentence_generation(model, t, '가는 말이', 4)) # 5번 예측


가는 말이 고와야 오는 말이 곱다


## 6.2 LSTM을 이용한 텍스트 생성하기

### 6.2.1 데이터 전처리

In [32]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [39]:
df=pd.read_csv('source/ArticlesApril2018.csv')
df.head(3)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...


"열의 개수가 굉장히 많기 때문에 한 눈에 보기 어렵습니다. 어떤 열이 있고, 열이 총 몇 개가 있는지 출력해봅시다.

In [37]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [40]:
df['headline'].isnull().values.any()

False

In [63]:
# Null 값은 별도로 없는 것으로 보입니다. headline 열에서 모든 신문 기사의 제목을 뽑아서 하나의 리스트로 저장해보도록 하겠습니다.

headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력
# 근데 null이어야 할 값에 unknow이 들어있다? 이런 경우 의심은 해봐야함(물론 다 검사는 함)

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [64]:
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

노이즈값 제거 후 샘플의 개수 : 1214


In [65]:
# 샘플의 수가 1,324에서 1,214로 110개의 샘플이 제거되었는데, 기존에 출력했던 5개의 샘플을 출력해보겠습니다.

headline[:5]


['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

<!-- 현재는 제거가 된 것을 확인하였습니다. 이제 데이터 전처리를 수행합니다. 여기서 선택한 전처리는 구두점 제거와 단어의 소문자화입니다. 전처리를 수행하고, 다시 샘플 5개를 출력합니다.
 -->
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

In [66]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 20


In [67]:
# 각 단어의 임베딩 벡터는 10차원을 가지고, 128의 은닉 상태 크기를 가지는 LSTM을 사용합니다. 
# 문장을 생성하는 함수 sentence_generation을 만들어서 문장을 생성해봅시다.

def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [68]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

C:\Users\mrjjo\anaconda3\envs\nlp38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


i 이 이 이 이 이 이 이 에 이 뛰


In [69]:
print(print(sentence_generation(model, t, 'how', 10)))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성)/

how 이 이 이 이 이 이 이 에 이 뛰
None


# 7. 글자 단위 RNN(Char RNN)

지금까지 배운 RNN은 전부 입력과 출력의 단위가 단어 벡터였습니다. 하지만 입출력의 단위를 단어 레벨(word-level)에서 글자 레벨(character-level)로 변경하여 RNN을 구현할 수 있습니다. 어려운 내용이 아니라 입, 출력의 단위를 단어에서 글자로 바꿨을 뿐입니다.



![python image2](https://wikidocs.net/images/page/48649/char_rnn1.PNG)

앞서 배운 단어 단위 RNN 언어 모델과 다른 점은 단어 단위가 아니라 글자 단위를 입, 출력으로 사용하므로 임베딩층(embedding layer)을 여기서는 사용하지 않겠습니다. 여기서는 언어 모델의 훈련 과정과 테스트 과정의 차이를 이해하는데 초점을 둡니다.

## 7.1 글자단위 RNN 언어모델

In [70]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

In [71]:
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")
f = open('11-0.txt', 'rb')
lines=[]
for line in f: # 데이터를 한 줄씩 읽는다.
    line=line.strip() # strip()을 통해 \r, \n을 제거한다.
    line=line.lower() # 소문자화.
    line=line.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(line) > 0:
        lines.append(line)
f.close()

In [74]:
# 간단한 전처리가 수행된 결과가 lines란 이름의 리스트에 저장되었습니다. 리스트에서 5개의 원소만 출력해보겠습니다.


lines[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at']

In [81]:
# 각 원소는 문자열로 구성되어져 있는데, 특별히 의미있게 문장 토큰화가 된 상태는 아닙니다. 이를 하나의 문자열로 통합하겠습니다.
text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(text))

문자열의 길이 또는 총 글자의 개수: 159484


In [89]:
print(text[:200])
print(text[:150])


the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with
the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and


In [90]:
text[:11]

'the project'

In [94]:
char_vocab = sorted(list(set(text))) # list(set())으로 유니크한 원소만 담은 리스트를 만들고, 정렬
vocab_size=len(char_vocab) # 그 리스트의 length
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 56


In [95]:
sorted(list(set(text)))

[' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [102]:
# 어떤 방대한 양의 텍스트라도 집합의 크기를 적게 가져갈 수 있다는 것은 
#구현과 테스트를 굉장히 쉽게 할 수 있다는 이점을 가지므로, RNN의 동작 메커니즘 이해를 위한 토이 프로젝트로 굉장히 많이 사용됩니다. 
#글자 집합에 인덱스를 부여하고 전부 출력해보겠습니다.

char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


In [118]:
for i, c in enumerate(char_vocab):
    print(c, i)

  0
! 1
" 2
# 3
$ 4
% 5
' 6
( 7
) 8
* 9
, 10
- 11
. 12
/ 13
0 14
1 15
2 16
3 17
4 18
5 19
6 20
7 21
8 22
9 23
: 24
; 25
? 26
[ 27
] 28
_ 29
a 30
b 31
c 32
d 33
e 34
f 35
g 36
h 37
i 38
j 39
k 40
l 41
m 42
n 43
o 44
p 45
q 46
r 47
s 48
t 49
u 50
v 51
w 52
x 53
y 54
z 55


In [120]:
# 이제 반대로 인덱스로부터 글자를 리턴하는 index_to_char을 만듭니다.
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [121]:
# 분리하는 방법은 문장 샘플의 길이를 정하고, 해당 길이만큼 문자열 전체를 전부 등분하는 것입니다.

seq_length = 60 # 문장의 길이를 60으로 한다.
n_samples = int(np.floor((len(text) - 1) / seq_length)) # 문자열을 60등분한다. 그러면 즉, 총 샘플의 개수
print ('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2658


In [122]:
# <!-- 만약 문장의 길이를 60으로 한다면 15만 8천을 60으로 나눈 수가 샘플의 수가 됩니다. 여기서는 총 샘플의 수가 2,646개입니다. -->

train_X = []
train_y = []

for i in range(n_samples): # 2,646번 수행
    X_sample = text[i * seq_length: (i + 1) * seq_length]
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 가져온다.
    X_encoded = [char_to_index[c] for c in X_sample] # 하나의 문장 샘플에 대해서 정수 인코딩
    train_X.append(X_encoded)

    y_sample = text[i * seq_length + 1: (i + 1) * seq_length + 1] # 오른쪽으로 1칸 쉬프트한다.
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [123]:
# 글자 단위 RNN에서는 입력 시퀀스에 대해서 워드 임베딩을 하지 않습니다. 
# 다시 말해 임베딩층(embedding layer)을 사용하지 않을 것이므로, 입력 시퀀스인 train_X에 대해서도 원-핫 인코딩을 합니다.

train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

In [124]:
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

train_X의 크기(shape) : (2658, 60, 56)
train_y의 크기(shape) : (2658, 60, 56)


![python image2](https://wikidocs.net/images/page/22886/rnn_image6between7.PNG)
이는 샘플의 수(No. of samples)가 2,646개, 입력 시퀀스의 길이(input_length)가 60, 각 벡터의 차원(input_dim)이 55임을 의미합니다.   
원-핫 벡터의 차원은 글자 집합의 크기인 55이어야 하므로 원-핫 인코딩이 수행되었음을 알 수 있습니다.

In [125]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

In [127]:
model = Sequential()
model.add(LSTM(256, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

In [129]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

Epoch 1/80
84/84 - 22s - loss: 3.0703 - accuracy: 0.1811
Epoch 2/80
84/84 - 24s - loss: 2.7933 - accuracy: 0.2288
Epoch 3/80
84/84 - 24s - loss: 2.4522 - accuracy: 0.3161
Epoch 4/80
84/84 - 24s - loss: 2.2950 - accuracy: 0.3506
Epoch 5/80
84/84 - 24s - loss: 2.1898 - accuracy: 0.3779
Epoch 6/80
84/84 - 24s - loss: 2.1078 - accuracy: 0.3977
Epoch 7/80
84/84 - 24s - loss: 2.0292 - accuracy: 0.4165
Epoch 8/80
84/84 - 24s - loss: 1.9647 - accuracy: 0.4321
Epoch 9/80
84/84 - 24s - loss: 1.9130 - accuracy: 0.4459
Epoch 10/80
84/84 - 24s - loss: 1.8655 - accuracy: 0.4600
Epoch 11/80
84/84 - 24s - loss: 1.8226 - accuracy: 0.4712
Epoch 12/80
84/84 - 24s - loss: 1.7820 - accuracy: 0.4841
Epoch 13/80
84/84 - 24s - loss: 1.7444 - accuracy: 0.4939
Epoch 14/80
84/84 - 24s - loss: 1.7078 - accuracy: 0.5028
Epoch 15/80
84/84 - 24s - loss: 1.6740 - accuracy: 0.5116
Epoch 16/80
84/84 - 24s - loss: 1.6406 - accuracy: 0.5201
Epoch 17/80
84/84 - 24s - loss: 1.6092 - accuracy: 0.5287
Epoch 18/80
84/84 - 24s

In [130]:
def sentence_generation(model, length):
    ix = [np.random.randint(vocab_size)] # 글자에 대한 랜덤 인덱스 생성
    y_char = [index_to_char[ix[-1]]] # 랜덤 익덱스로부터 글자 생성
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')
    X = np.zeros((1, length, vocab_size)) # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성

    for i in range(length):
        X[0][i][ix[-1]] = 1 # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)
sentence_generation(model, 100)

3 번 글자 # 로 예측을 시작!
# the production, promotion and distribution of project gutenberg-tm work. 1.e.9. you may for a reel

'# the production, promotion and distribution of project gutenberg-tm work. 1.e.9. you may for a reely'

## 7.2 글자 단위 RNN(Char RNN)으로 텍스트 생성하기
이번에는 다 대 일(many-to-one) 구조의 RNN을 글자 단위로 학습시키고, 텍스트 생성을 해보겠습니다.



In [131]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# 다음과 같이 제가 임의로 만든 엉터리 노래 가사가 있습니다.

text='''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

In [132]:
# 우선 위의 텍스트에 존재하는 단락 구분을 없애고 하나의 문자열로 재저장하겠습니다.

tokens = text.split() # '\n 제거'
text = ' '.join(tokens)
print(text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [133]:
char_vocab = sorted(list(set(text))) # 중복을 제거한 글자 집합 생성
print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [134]:
#기존의 단어 단위의 집합이 아니라 알파벳 또는 구두점 등의 단위의 집합인 글자 집합이 생성되었습니다.


vocab_size=len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 33


In [135]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


In [136]:
# 여기서는 입력 시퀀스의 길이. 즉, 모든 샘플들의 길이가 10가 되도록 데이터를 구성해보겠습니다. 
# 예측 대상이 되는 글자도 필요하므로 우선 길이가 11이 되도록 데이터를 구성합니다.

length = 11
sequences = []
for i in range(length, len(text)):
    seq = text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


In [137]:
# 총 샘플의 수는 426개가 완성되었습니다. 이 중 10개만 출력해보겠습니다.

sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

In [138]:
X = []
for line in sequences: # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다.
    temp_X = [char_to_index[char] for char in line] # 문장 샘플에서 각 글자에 대해서 정수 인코딩을 수행.
    X.append(temp_X)
# 정수 인코딩 된 결과가 X에 저장되었습니다. 5개만 출력해보겠습니다.

for line in X[:5]:
    print(line)

[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18]
[0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28]
[16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17]
[14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0]
[28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]


In [139]:
# 정상적으로 정수 인코딩이 수행되었습니다. 이제 예측 대상인 글자를 분리시켜주는 작업을 합니다. 
# 모든 샘플 문장에 대해서 맨 마지막 글자를 분리시켜줍니다.

sequences = np.array(X)
X = sequences[:,:-1]
y = sequences[:,-1] # 맨 마지막 위치의 글자를 분리

In [140]:
# 정상적으로 분리가 되었는지 X와 y 둘 다 5개씩 출력해보겠습니다.

for line in X[:5]:
    print(line)

[ 8  0 16 14 28  0 24 23  0 31]
[ 0 16 14 28  0 24 23  0 31 18]
[16 14 28  0 24 23  0 31 18 28]
[14 28  0 24 23  0 31 18 28 17]
[28  0 24 23  0 31 18 28 17  0]


In [141]:
print(y[:5])


[18 28 17  0 21]


In [142]:
# 앞서 출력한 5개의 샘플에서 각각 맨 뒤의 글자였던 18, 28, 17, 0, 21이 별도로 분리되어 y에 저장되었습니다. 이제 X와 y에 대해서 원-핫 인코딩을 수행해보겠습니다.

sequences = [to_categorical(x, num_classes=vocab_size) for x in X] # X에 대한 원-핫 인코딩
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size) # y에 대한 원-핫 인코딩
# 원-핫 인코딩이 수행되었는지 확인하기 위해 수행한 후의 X의 크기(shape)를 보겠습니다.

print(X.shape)

(426, 10, 33)


In [143]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
model = Sequential()
model.add(LSTM(80, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

In [144]:
# LSTM을 사용하고, 은닉 상태의 크기는 80, 그리고 출력층에 단어 집합의 크기만큼의 뉴런을 배치하여 모델을 설계합니다.

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
14/14 - 1s - loss: 3.4529 - accuracy: 0.1455
Epoch 2/100
14/14 - 0s - loss: 3.2846 - accuracy: 0.1972
Epoch 3/100
14/14 - 0s - loss: 3.0269 - accuracy: 0.1972
Epoch 4/100
14/14 - 0s - loss: 2.9736 - accuracy: 0.1972
Epoch 5/100
14/14 - 0s - loss: 2.9455 - accuracy: 0.1972
Epoch 6/100
14/14 - 0s - loss: 2.9342 - accuracy: 0.1972
Epoch 7/100
14/14 - 0s - loss: 2.9181 - accuracy: 0.1972
Epoch 8/100
14/14 - 0s - loss: 2.8938 - accuracy: 0.1972
Epoch 9/100
14/14 - 0s - loss: 2.8652 - accuracy: 0.1972
Epoch 10/100
14/14 - 0s - loss: 2.8380 - accuracy: 0.1972
Epoch 11/100
14/14 - 0s - loss: 2.8144 - accuracy: 0.1972
Epoch 12/100
14/14 - 0s - loss: 2.7794 - accuracy: 0.1972
Epoch 13/100
14/14 - 0s - loss: 2.7180 - accuracy: 0.2019
Epoch 14/100
14/14 - 0s - loss: 2.6812 - accuracy: 0.2371
Epoch 15/100
14/14 - 0s - loss: 2.6227 - accuracy: 0.2371
Epoch 16/100
14/14 - 0s - loss: 2.5753 - accuracy: 0.2394
Epoch 17/100
14/14 - 0s - loss: 2.5408 - accuracy: 0.2629
Epoch 18/100
14/14 - 0s

In [145]:
# 문장을 생성하는 함수 sentence_generation을 만들어서 문장을 생성해봅시다.

def sentence_generation(model, char_to_index, seq_length, seed_text, n):
# 모델, 인덱스 정보, 문장 길이, 초기 시퀀스, 반복 횟수
    init_text = seed_text # 문장 생성에 사용할 초기 시퀀스
    sentence = ''

    for _ in range(n): # n번 반복
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))
        result = model.predict_classes(encoded, verbose=0)
        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장.
        for char, index in char_to_index.items(): # 만약 예측한 글자와 인덱스와 동일한 글자가 있다면
            if index == result: # 해당 글자가 예측 글자이므로 break
                break
        seed_text=seed_text + char # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경
        sentence=sentence + char # 예측 글자를 문장에 저장
        # for문이므로 이 작업을 다시 반복

    sentence = init_text + sentence
    return sentence
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on with life as a programmer, I like to hang out with programming and deep learning.
